In [1]:
!pip install -q transformers==4.31.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 42.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 88.3 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kaggle-environments 1.16.11 requires transformers>=4.33.1, but you have transformers 4.31.0 which is incompatible.
sentence-transformers 3.3.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.31.0 which is incompatible.


In [2]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.41.1 transformers==4.31.0 trl==0.4.7

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 5.7 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 18.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 5.5 MB/s eta 0:00:00


In [3]:
!pip install -U bitsandbytes accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 22.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 25.9 MB/s eta 0:00:00
  Attempting uninstall: bitsandbytes
    Found existing installation: bitsandbytes 0.41.1
    Uninstalling bitsandbytes-0.41.1:
      Successfully uninstalled bitsandbytes-0.41.1
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.21.0
    Uninstalling accelerate-0.21.0:
      Successfully uninstalled accelerate-0.21.0


In [4]:
import bitsandbytes as bnb
import accelerate
import transformers
import trl
import peft

print("bitsandbytes:", bnb.__version__)
print("accelerate:", accelerate.__version__)
print("transformers:", transformers.__version__)
print("trl:", trl.__version__)
print("peft:", peft.__version__)


bitsandbytes: 0.45.4
accelerate: 1.6.0
transformers: 4.31.0
trl: 0.4.7
peft: 0.4.0


In [5]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [6]:
# The model that you want to train from the Hugging Face hub
model_name = "NousResearch/Llama-2-7b-hf"

# Fine-tuned model name
new_model = "Llama-2-7b-chat-finetune"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs (increase from 1 to 5)
num_train_epochs = 10

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25
################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [7]:
import pandas as pd

# Load your CSV
df = pd.read_csv("/kaggle/input/dataset11/Q2_dataset_fazul.csv")

# Convert non-string columns to string before concatenation
df["Year"] = df["Year"].astype(str)
df["Paper_Varient"] = df["Paper_Varient"].astype(str)

# Create a single text column with all the info
df["text"] = (
    "Year: " + df["Year"] + "\n" +
    "Paper_Session: " + df["Paper_Session"] + "\n" +
    "Paper_Varient: " + df["Paper_Varient"] + "\n" +
    "Q2_Topic: " + df["Q2_Topic"] + "\n" +
    "Q2: " + df["Q2"]
)

# Now df["text"] holds your combined text
print(df["text"].iloc[0])


Year: 2023
Paper_Session: ON
Paper_Varient: 11
Q2_Topic: the major themes of the Qur’an as contained both in the passages set for special study and in other similar passages
Q2: (a) Using the set passages you have studied, write about God’s relationship with the created world.  (b) What do these passages teach Muslims about responsibility towards their environment?


In [8]:


# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training



# Inspect the first combined text
print("SAMPLE TEXT:\n", df["text"].iloc[0])
texts = df["text"].tolist()


# -----------------------
# 2. Tokenize
# -----------------------

# We'll create a simple custom dataset class that returns tokenized input.
class Q2Dataset(torch.utils.data.Dataset):
    def __init__(self, texts, tokenizer, max_length=512):
        self.examples = []
        for t in texts:
            # Tokenize each text with truncation
            tokenized = tokenizer(
                t,
                truncation=True,
                max_length=max_length,
                return_tensors="pt"
            )
            self.examples.append({
                "input_ids": tokenized["input_ids"].squeeze(0),
                "attention_mask": tokenized["attention_mask"].squeeze(0),
            })

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        return self.examples[idx]  # ✅ Properly indented


dataset = Q2Dataset(texts, tokenizer, max_length=512)

# You could split into train/test if you want. For small data, we’ll skip that:
train_dataset = dataset

print("Number of training samples:", len(train_dataset))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

SAMPLE TEXT:
 Year: 2023
Paper_Session: ON
Paper_Varient: 11
Q2_Topic: the major themes of the Qur’an as contained both in the passages set for special study and in other similar passages
Q2: (a) Using the set passages you have studied, write about God’s relationship with the created world.  (b) What do these passages teach Muslims about responsibility towards their environment?
Number of training samples: 28


In [9]:
# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Train model
trainer.train()

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_r

Step,Training Loss
25,1.681900
50,0.632000


TrainOutput(global_step=70, training_loss=0.9512085778372629, metrics={'train_runtime': 547.3007, 'train_samples_per_second': 0.512, 'train_steps_per_second': 0.128, 'total_flos': 778065218273280.0, 'train_loss': 0.9512085778372629, 'epoch': 10.0})

In [12]:
# Save trained model
trainer.model.save_pretrained(new_model)

In [14]:
prompt = """
You are an O-level exam question generator that will predict new question based on pattern depicted in datatset.
Please output the question in the following format:
(a)
(b)

Only output the question text, nothing else.

(a) Describe how the Qur’an addresses the concept of God’s oneness.
(b) Explain why understanding this concept is essential for a Muslim’s faith.

Now create a **new** question in the same (a)/(b) format, focusing on the major themes of the Qur’an:
(a)
(b)
"""

# Use the LoRA-updated model
peft_model = trainer.model
peft_model.eval()

inputs = tokenizer(prompt, return_tensors="pt").to(peft_model.device)
with torch.no_grad():
    output_ids = peft_model.generate(
        **inputs,
        max_length=300,
        temperature=0.7,
        top_p=0.9,
        do_sample=True
    )

generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print("Generated question:")
print(generated_text)


Generated question:

You are an O-level exam question generator that will predict new question based on pattern depicted in datatset.
Please output the question in the following format:
(a)
(b)

Only output the question text, nothing else.

(a) Describe how the Qur’an addresses the concept of God’s oneness.
(b) Explain why understanding this concept is essential for a Muslim’s faith.

Now create a **new** question in the same (a)/(b) format, focusing on the major themes of the Qur’an:
(a)
(b)

Only output the question text, nothing else.

(a) The Qur’an describes God’s creation of the heavens and the earth. Why do you think it is important for Muslims to know about this?
(b) How do Muslims use this knowledge to understand God’s power and greatness?

### Exercise:

(a) Write your own question in the format above.
(b) Explain why the Qur’an is important for Muslims to study and understand.






